# Data Scraping and Sentiment Analysis

In [ ]:
!pip install gnews
!pip install newspaper3k
!pip3 install pandas
!pip3 install --upgrade openai

In [ ]:
from gnews import GNews
import csv
from datetime import datetime, timedelta

google_news = GNews()
# google_news.max_results = 5  # number of responses across a keyword
google_news.country = 'United States'  # News from a specific country
google_news.language = 'english'  # News in a specific language

# Set the start and end dates for the 10-year period
start_date = datetime(2014, 1, 1)
end_date = datetime(2014, 1, 6)

errors = []
# Open a CSV file to save the results
with open('microsoft_news.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Date', 'Publisher', 'Published_Date', 'Title', 'Description', 'URL', 'Text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over each day in the 10-year period
    current_date = start_date
    
    print(f"Searching Articles in {current_date}")
    google_news.start_date = (current_date.year, current_date.month, current_date.day)
    next_date = current_date + timedelta(days=1)
    google_news.end_date = (end_date.year, end_date.month, end_date.day)

    microsoft_news = google_news.get_news('Microsoft')

    # Iterate over the available articles for the current day
    for article_info in microsoft_news:
        article_data = {
            'Date': current_date.strftime('%Y-%m-%d'),
            'Publisher': article_info.get('publisher', '').get('title', ''),
            'Published_Date': article_info.get('published date', ''),
            'Title': article_info.get('title', ''),
            'Description': article_info.get('description', ''),
            'URL': article_info.get('url', '')
        }
        
        try:
            article = google_news.get_full_article(article_info['url'])
            article_data['Text'] = article.text
            
        except Exception as e:
            print(f"Error retrieving article: {e}")
            errors.append(e)
            continue
        writer.writerow(article_data)

        

print("News scraping and CSV export completed.")

## Scraping Google news

In [ ]:
from gnews import GNews
import csv
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_articles_for_week(start_date, end_date):
    print(f"Fetching Articles from {start_date} to {end_date}")
    local_google_news = GNews()
    local_google_news.country = 'United States'
    local_google_news.language = 'english'
    local_google_news.start_date = (start_date.year, start_date.month, start_date.day)
    local_google_news.end_date = (end_date.year, end_date.month, end_date.day)
    microsoft_news = local_google_news.get_news('Microsoft')

    articles_data = []
    for article_info in microsoft_news:
        article_data = {
            'Date': start_date.strftime('%Y-%m-%d') + " to " + end_date.strftime('%Y-%m-%d'),
            'Publisher': article_info.get('publisher', '').get('title', ''),
            'Published_Date': article_info.get('published date', ''),
            'Title': article_info.get('title', ''),
            'Description': article_info.get('description', ''),
            'URL': article_info.get('url', '')
        }
        try:
            article = local_google_news.get_full_article(article_info['url'])
            article_data['Text'] = article.text
        except Exception as e:
            print(f"Error retrieving article for {start_date} to {end_date}: {e}")
            continue
        articles_data.append(article_data)
    return articles_data

def week_ranges(start_date, end_date):
    delta = timedelta(days=1)
    while start_date < end_date:
        week_end = start_date + timedelta(days=6)
        if week_end > end_date:
            week_end = end_date
        yield (start_date, week_end)
        start_date += timedelta(days=7)

start_date = datetime(2014, 1, 1)
end_date = datetime(2024, 1, 1)

# Open CSV file for writing results
fieldnames = ['Date', 'Publisher', 'Published_Date', 'Title', 'Description', 'URL', 'Text']
with open('microsoft_decade_news.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(fetch_articles_for_week, start, end) for start, end in week_ranges(start_date, end_date)]
        for future in as_completed(futures):
            articles_data = future.result()
            for article_data in articles_data:
                writer.writerow(article_data)

print("Weekly news scraping and CSV export completed.")

## News Sentiment Analysis

In [ ]:
import csv
from datetime import datetime, timedelta
import openai
import pandas as pd
from openai import OpenAI
import tiktoken
# Set up OpenAI API key
OPENAI_API_KEY=""
client = openai.OpenAI(api_key=OPENAI_API_KEY)
# Function to analyze sentiment using OpenAI API
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
def analyze_sentiment(text):
    # Define a rough maximum character length based on average token size
    max_characters = 16000 * 4  # Assuming an average of 4 characters per token

    # Truncate the text if it's longer than the maximum character length
    while len(encoder.encode(text)) > 16000:
        text = text[:max_characters]
        max_characters -= 1000
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        
        messages=[
                {"role": "system", "content": "You are a stock market expert, skilled in analyzing a batch of news for a company and giving a sentiment for its affect on the stock of the company."},
                {"role": "user", "content": f"(IMPORTANT! ONLY REPLY WITH A NUMBER)\nAnalyze the sentiment of the following batch of news articles about Microsoft and provide a rating from 1-100 about its effect on the company's stock (1 being most negative, 100 being most positive). if you give a rating of 100 then that means these news will tremendously affect the stock positively and increase the price. if you give a rating of 0 then that means these news will tremendously affect the stock negatively and decrease the price. If you give a rating of 50 then that means these news will not affect the stock. Here are the articles:\n\n{text}\n\nSentiment rating (IMPORTANT! ONLY REPLY WITH A NUMBER):"}
            ]
        ,
        temperature=0.8,
        max_tokens=1,
        
    )
    sentiment_rating = response.choices[0].message.content
    return sentiment_rating

# Read in the news dataset
df = pd.read_csv("microsoft_decade_news.csv",  encoding='utf-8')

df["Date"] = pd.to_datetime(df["Published_Date"]).dt.date  # Ensure Date is in datetime.date format for accurate grouping

df = df.sort_values("Date")
# Group by date and select up to 5 articles per date

grouped = df.groupby("Date")
selected_articles = grouped.apply(lambda x: x.head(10)).reset_index(drop=True)

# Open a CSV file to save the results
with open('news_sentiment_MSFT.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Date', 'Sentiment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over each day in the 10-year period
    for date, group in selected_articles.groupby("Date"):
        print(f"Processing: {date}")
        
        # Concatenate 10 articles for the current day
        articles_text = '\n'.join(f"Article {i}\nTitle: {row.Title}\nText: {row.Text}\n##################################\n" for i, row in enumerate(group.itertuples(), 1))

        sentiment_rating = analyze_sentiment(articles_text)
   
        # Write the article details and sentiment rating to the CSV file
        writer.writerow({
            'Date': date,
            'Sentiment': sentiment_rating
        })        

print("News scraping, sentiment analysis, and CSV export completed.")

### Function to calculate cost
you have to integrate this with the previous cell to use
replace "analyze_sentiment(articles_text)" with "calculate_cost(articles_text)" and sum the cost

In [45]:
import tiktoken
import pandas as pd

price_token = 0.5
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

def calculate_cost(text):
    max_characters = 16000 * 4
    while len(encoder.encode(text)) > 16000:
        text = text[:max_characters]
        max_characters -= 1000
    num_tokens = len(encoder.encode(text))
    cost = num_tokens / 1000000 * price_token
    return cost



In [12]:
df = pd.read_csv("microsoft_news_sentiment.csv", encoding="utf-8")

df = df.sort_values("Date")

df.to_csv("news_sentiment_MSFT.csv", encoding="utf-8")

## Reddit posts
Data downloaded from pushshift

In [ ]:
import pandas as pd
df1 = pd.read_csv("wallstreetbets_submissions.csv")
df2 = pd.read_csv("wallstreetbets__submissions.csv")

In [ ]:
df_combined = pd.concat([df1, df2])
df_combined.drop_duplicates(inplace=True)
df_combined = df_combined.sort_values("created", ignore_index=True)

df_combined.to_csv("wallstreetbets.csv", index=False)

In [ ]:
df = pd.read_csv("wallstreetbets.csv")
df

,author,title,score,created,link,text,url
0,u/[deleted],Earnings season is here. Place your bets.,13,2012-04-12 00:40,https://www.reddit.com/r/wallstreetbets/commen...,I know that /r/investing is a great place for ...,http://www.reddit.com/r/wallstreetbets/comment...
1,u/[deleted],"GOOG - beat estimates, price barely rises.",2,2012-04-13 04:37,https://www.reddit.com/r/wallstreetbets/commen...,NaN,http://www.bloomberg.com/news/2012-04-12/googl...
2,u/[deleted],My poorly timed opening position for AAPL earn...,12,2012-04-17 06:29,https://www.reddit.com/r/wallstreetbets/commen...,"So I missed out on GOOG, which is probably a g...",http://www.reddit.com/r/wallstreetbets/comment...
3,u/[deleted],Anyone betting on VVUS and their potential app...,1,2012-04-17 21:41,https://www.reddit.com/r/wallstreetbets/commen...,"I'm normally a long, but I've created a second...",http://www.reddit.com/r/wallstreetbets/comment...
4,u/secondhandsondek,"EBAY posts higher 1Q net income and revenue, s...",7,2012-04-19 22:10,https://www.reddit.com/r/wallstreetbets/commen...,NaN,http://imgur.com/aAfCi
...,...,...,...,...,...,...,...
2359721,u/ThtGrlUDntKno,Are small Calls/Puts worth it?,1,2024-01-01 07:20,https://www.reddit.com/r/wallstreetbets/commen...,[removed],https://www.reddit.com/r/wallstreetbets/commen...
2359722,u/Exegi_One,Future Trading BTC/USDT,1,2024-01-01 07:28,https://www.reddit.com/r/wallstreetbets/commen...,NaN,https://i.redd.it/uvnlbm82tp9c1.jpeg
2359723,u/FinanceHubNow,"Netflix Gave Director 66,000,000 to Make A Sho...",1,2024-01-01 07:28,https://www.reddit.com/r/wallstreetbets/commen...,[removed],https://www.reddit.com/r/wallstreetbets/commen...
2359724,u/coco88888,New Year's Eve,211,2024-01-01 07:45,https://www.reddit.com/r/wallstreetbets/commen...,NaN,https://i.redd.it/8rbmtol6wp9c1.jpeg


In [ ]:
# Filter rows where 'text' or 'title' contain 'microsoft' or 'msft'
filtered_df = df[(df['text'].str.contains('microsoft|msft', case=False)) | 
                 (df['title'].str.contains('microsoft|msft', case=False))]
filtered_df.to_csv("wallstreetbets_MSFT_Microsoft_filtered.csv", index=False)

## For Reddit sentiment

In [ ]:
import csv
from datetime import datetime, timedelta
import openai
import pandas as pd
from openai import OpenAI
import tiktoken
# Set up OpenAI API key
OPENAI_API_KEY=""
client = openai.OpenAI(api_key=OPENAI_API_KEY)
# Function to analyze sentiment using OpenAI API
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
def analyze_sentiment(text):
    # Define a rough maximum character length based on average token size
    max_characters = 16000 * 4  # Assuming an average of 4 characters per token

    # Truncate the text if it's longer than the maximum character length
    while len(encoder.encode(text)) > 16000:
        text = text[:max_characters]
        max_characters -= 1000

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a stock market expert, skilled in analyzing a batch of social media posts on reddit and giving a sentiment for its affect on the stock of the company."},
                {"role": "user", "content": f"(IMPORTANT! ONLY REPLY WITH A NUMBER)\nAnalyze the sentiment of the following batch of Reddit posts about Microsoft and provide a rating from 1-100 about its effect on the company's stock (1 being most negative, 100 being most positive). if you give a rating of 100 then that means these posts will tremendously affect the stock positively and increase the price. if you give a rating of 0 then that means these posts will tremendously affect the stock negatively and decrease the price. If you give a rating of 50 then that means these posts will not affect the stock. Here are the posts:\n\n{text}\n\nSentiment rating (IMPORTANT! ONLY REPLY WITH A NUMBER):"}
            ]
        ,
        temperature=0.8,
        max_tokens=1,
        
    )
    sentiment_rating = response.choices[0].message.content
    return sentiment_rating

# Read in the reddit dataset
df = pd.read_csv("wallstreetbets_MSFT_Microsoft_filtered.csv")

df["Date"] = pd.to_datetime(df["created"]).dt.date  # Ensure Date is in datetime.date format for accurate grouping
# Filter to keep only dates from 2014-2023
df = df[df["Date"] >= pd.to_datetime("2014-01-01").date()]
df = df[df["Date"] < pd.to_datetime("2024-01-01").date()]
df = df.sort_values("Date")

# Group by date and select up to 5 articles per date

grouped = df.groupby("Date")
selected_articles = grouped.apply(lambda x: x.head(1000)).reset_index(drop=True)

# Open a CSV file to save the results
with open('microsoft_reddit_sentiment.csv', 'w', newline='', encoding="utf-8") as csvfile:
    fieldnames = ['Date', 'Sentiment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over each day in the 10-year period
    for date, group in selected_articles.groupby("Date"):
        print(f"Processing: {date}")
        
        # Concatenate the posts for the current day
        posts = '\n'.join(f"Article {i}\nUpvotes: {row.score}\nTitle: {row.title}\nText: {row.text}\n##################################\n" for i, row in enumerate(group.itertuples(), 1))

        sentiment_rating = analyze_sentiment(posts)
   
        # Write the article details and sentiment rating to the CSV file
        writer.writerow({
            'Date': date,
            'Sentiment': sentiment_rating
        })        

print("Reddit sentiment analysis, and CSV export completed.")
# sum

In [13]:
df = pd.read_csv("microsoft_reddit_sentiment.csv", encoding="utf-8")

df = df.sort_values("Date")
df.to_csv("Reddit_sentiment_MSFT.csv", encoding="utf-8", index=False)